In [4]:
from scipy import interpolate
import numpy as np
import ast
import csv
import matplotlib.pyplot as plt
import pylcp
import pickle
import scipy.constants as consts

In [14]:
class ComsolMagCylindricalInterpolation(object):
    """
    Class to interpolate 2D comsol field data. Assumed cylindrically symmetric and sampled over a rectangular grid. The grids used for radial and axial components should be the same

    Parameters
    ----------
    
    radialFile
    axialFile
    .csv files of the form (r,z,B_i)
    
    gammaMHz
    laser cooling transition linewidth, for rescaling. Give in MHz, i.e. megacycles per second
   
    Attributes
    ----------
        
    radialSamplePoints :
    1D array of the radial sampling coordinates
    
    axialSamplePoints
    1D array for the axial sampling coordiantes
    
    radialComponents :
    radial field component in G
    
    """

    def __init__(self, radialFile,axialFile,gammaMHz=1,symmetry=1, gauss_conv = lambda self : 10**(-4)*consts.value('Bohr magneton')/(10**6*self.gammaMHz*consts.h), cm_conv = 1):
        
        self.cm_unit = cm_conv
        self.gammaMHz = gammaMHz
        self.symmetry = symmetry
        self.gaussToGammaCalibration = gauss_conv
        with open(radialFile, 'r') as csvfile:
            reader = csv.reader(csvfile, delimiter=',', quoting=csv.QUOTE_NONE)
            strData =  [row for row in reader]

        self.bRadHeader = strData[0:8]
        
        self.bRaddata = np.array([[float(i) for i in row] for row in strData[9:]])
        self.radialSamplePoints = 100*np.unique([i[0] for i in self.bRaddata])
        self.axialSamplePoints = 100*np.unique([i[1] for i in self.bRaddata])
        self.bRad= interpolate.RectBivariateSpline(self.radialSamplePoints,self.axialSamplePoints,self.symmetry*np.transpose(np.array(np.array_split([k[2] for k in self.bRaddata],len(self.axialSamplePoints)))),s=0, kx = 1,ky=1)
        
        with open(axialFile, 'r') as csvfile:
            reader = csv.reader(csvfile, delimiter=',', quoting=csv.QUOTE_NONE)
            strData =  [row for row in reader]
        
        self.bAxHeader = strData[0:8]
        
        self.bAxdata = np.array([[float(i) for i in row] for row in strData[9:]])
        self.bAx= interpolate.RectBivariateSpline(self.radialSamplePoints,self.axialSamplePoints,self.symmetry*np.transpose(np.array(np.array_split([k[2] for k in self.bAxdata],len(self.axialSamplePoints)))),s=0, kx = 1,ky=1)
        
    def gaussToGammaCalibration(self):
        raise ValueError("Class not initialised correctly")
        
    def fieldCartesian(self, r,t):
        #Here is a function that pylcp should be able to use. I project the radial field components onto the x- and y-axes for use with Cartesians.
        #For pylcp to use it, it is better to keep the function arguments as R, t "(promote_to_lambda seems to like finding the function signatures this way to test it)"
        r_cm = r*self.cm_unit
        if(abs(r_cm[0]) > np.amax(self.radialSamplePoints) or abs(r_cm[1]) > np.amax(self.radialSamplePoints) or abs(r_cm[2]) > np.amax(self.axialSamplePoints)):
            return self.gaussToGammaCalibration()*np.array([0,0,1e-6])
        rho = np.sqrt(r_cm[0]**2 + r_cm[1]**2)
        Bz = self.bAx(rho,r_cm[2])
        if (rho==0):
            Bx=0
            By=0
        else:
            Bx = self.bRad(rho,r_cm[2])*r_cm[0]/rho
            By = self.bRad(rho,r_cm[2])*r_cm[1]/rho
        return self.gaussToGammaCalibration()*np.ndarray.flatten(np.array([Bx,By,Bz]))

In [1]:
class ComsolMag2DCartesianInterpolation(object):
    """
    Class to interpolate 2D comsol field data. Assumed cartesian coordiantes and sampled over a rectangular grid in the XY plane. The grids used for each direction should be the same size.

    Parameters
    ----------
    
    BxFile
    ByFile
    BzFile
    
    .csv files of the form (x,y,B_i)
    
    gammaMHz
    laser cooling transition linewidth, for rescaling. Give in MHz, i.e. megacycles per second
    
    symmtery
    should be +/-1 to change the overall field direction
   
    Attributes
    ----------
        
    x SamplePoints :
    1D array of the x sampling coordinates
    
    y SamplePoints
    1D array for the y sampling coordiantes
    
    bZ :
    z-component of B in G
    
    bY :
    y-component of B in G
    
    bX :
    x-component of B in G
    
    
    """
    def __init__(self, bXFile,bYFile,bZFile,gammaMHz=1,symmetry=1):
        
        self.gammaMHz = gammaMHz
        self.symmetry = symmetry
        with open(bXFile, 'r') as csvfile:
            reader = csv.reader(csvfile, delimiter=',', quoting=csv.QUOTE_NONE)
            strData =  [row for row in reader]

        self.bXHeader = strData[0:8]
        
        self.bXdata = np.array([[float(i) for i in row] for row in strData[9:]])
        self.xSamplePoints = 100*np.unique([i[0] for i in self.bXdata])
        self.ySamplePoints = 100*np.unique([i[1] for i in self.bXdata])
        self.bX= interpolate.RectBivariateSpline(self.xSamplePoints,self.ySamplePoints,self.symmetry*np.array(np.array_split([k[2] for k in self.bXdata],len(self.ySamplePoints))),s=0, kx = 1,ky=1)
        
        with open(bYFile, 'r') as csvfile:
            reader = csv.reader(csvfile, delimiter=',', quoting=csv.QUOTE_NONE)
            strData =  [row for row in reader]
        
        self.bYHeader = strData[0:8]
        
        self.bYdata = np.array([[float(i) for i in row] for row in strData[9:]])
        self.bY= interpolate.RectBivariateSpline(self.xSamplePoints,self.ySamplePoints,self.symmetry*np.array(np.array_split([k[2] for k in self.bYdata],len(self.ySamplePoints))),s=0, kx = 1,ky=1)
        
        with open(bZFile, 'r') as csvfile:
            reader = csv.reader(csvfile, delimiter=',', quoting=csv.QUOTE_NONE)
            strData =  [row for row in reader]
        
        self.bZHeader = strData[0:8]
        
        self.bZdata = np.array([[float(i) for i in row] for row in strData[9:]])
        self.bZ= interpolate.RectBivariateSpline(self.xSamplePoints,self.ySamplePoints,self.symmetry*np.array(np.array_split([k[2] for k in self.bZdata],len(self.ySamplePoints))),s=0, kx = 1,ky=1)
        
    def gaussToGammaCalibration(self):
        return 10**(-4)*consts.value('Bohr magneton')/(10**6*self.gammaMHz*consts.h)
        
    def fieldCartesian(self, r,t):
        #Here is a function that pylcp should be able to use.
        Bx = self.bX(r[0],r[1])
        By = self.bY(r[0],r[1])
        Bz= self.bZ(r[0],r[1])
        return self.gaussToGammaCalibration()*np.ndarray.flatten(np.array([Bx,By,Bz]))